In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
import pandas as pd
x_train= pd.read_csv('/content/X_train_NHkHMNU.csv')
y_train = pd.read_csv("/content/y_train_ZAN5mwg.csv")
x_test = pd.read_csv('/content/X_test_final.csv')
y_random = pd.read_csv('/content/y_test_random_final.csv')

# Describing the datasets to check for missing values
dataframes_info = {
    'X_train': x_train.describe(include='all'),
    'Y_train': y_train.describe(include='all'),
    'X_test': x_test.describe(include='all'),
    'Y_random': y_random.describe(include='all')
}

# Outputting the descriptions
for name, df_info in dataframes_info.items():
    print(f'{name} Summary:\n', df_info)
    print(f'{name} Missing Values:\n', df_info.isnull().sum())

X_train Summary:
                  ID       DAY_ID COUNTRY  DE_CONSUMPTION  FR_CONSUMPTION  \
count   1494.000000  1494.000000    1494     1494.000000     1494.000000   
unique          NaN          NaN       2             NaN             NaN   
top             NaN          NaN      FR             NaN             NaN   
freq            NaN          NaN     851             NaN             NaN   
mean    1072.759036   591.861446     NaN        0.427442       -0.020032   
std      618.013179   345.065043     NaN        0.673412        0.918995   
min        0.000000     0.000000     NaN       -2.265563       -1.462350   
25%      540.250000   292.250000     NaN       -0.037421       -0.716771   
50%     1077.500000   591.000000     NaN        0.357061       -0.394166   
75%     1597.500000   885.750000     NaN        0.922057        0.650533   
max     2146.000000  1215.000000     NaN        2.033851        3.300640   

        DE_FR_EXCHANGE  FR_DE_EXCHANGE  DE_NET_EXPORT  FR_NET_EXPORT 

In [4]:
x_train

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,1054,206,FR,0.210099,-0.427458,-0.606523,0.606523,NaN,0.692860,NaN,...,-0.444661,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445
1,2049,501,FR,-0.022399,-1.003452,-0.022063,0.022063,-0.573520,-1.130838,0.573520,...,-1.183194,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365
2,1924,687,FR,1.395035,1.978665,1.021305,-1.021305,-0.622021,-1.682587,0.622021,...,1.947273,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952
3,297,720,DE,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,...,-0.976974,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948
4,1101,818,FR,0.143807,-0.617038,-0.924990,0.924990,NaN,0.990324,NaN,...,-0.526267,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,459,809,DE,1.529204,1.106682,-1.855327,1.855327,-0.218658,1.450426,0.218658,...,0.509514,NaN,NaN,NaN,NaN,NaN,NaN,0.876984,0.819520,1.320373
1490,1674,887,FR,1.618582,1.752840,0.611392,-0.611392,0.449153,-0.152146,-0.449153,...,1.666252,NaN,NaN,NaN,NaN,NaN,NaN,0.932633,-0.085690,0.356356
1491,748,1083,DE,0.856399,0.489199,-0.255778,0.255778,-1.531544,-0.829568,1.531544,...,0.358120,0.207905,0.404763,-0.594595,0.894011,0.256338,0.402316,-1.112899,-0.237835,0.067152
1492,1454,1133,FR,0.560689,-0.343777,-0.830239,0.830239,-0.304856,1.210230,0.304856,...,-0.184862,-0.682815,-0.390304,-0.972088,-1.501930,1.215528,1.338708,0.962812,-5.392852,-0.843812


In [5]:
# Filling all missing values with 0 for each dataframe
x_train.fillna(0, inplace=True)
y_train.fillna(0, inplace=True)
x_test.fillna(0, inplace=True)
y_random.fillna(0, inplace=True)


print(x_train.isnull().sum())
print(y_train.isnull().sum())
print(x_test.isnull().sum())
print(y_random.isnull().sum())


ID                  0
DAY_ID              0
COUNTRY             0
DE_CONSUMPTION      0
FR_CONSUMPTION      0
DE_FR_EXCHANGE      0
FR_DE_EXCHANGE      0
DE_NET_EXPORT       0
FR_NET_EXPORT       0
DE_NET_IMPORT       0
FR_NET_IMPORT       0
DE_GAS              0
FR_GAS              0
DE_COAL             0
FR_COAL             0
DE_HYDRO            0
FR_HYDRO            0
DE_NUCLEAR          0
FR_NUCLEAR          0
DE_SOLAR            0
FR_SOLAR            0
DE_WINDPOW          0
FR_WINDPOW          0
DE_LIGNITE          0
DE_RESIDUAL_LOAD    0
FR_RESIDUAL_LOAD    0
DE_RAIN             0
FR_RAIN             0
DE_WIND             0
FR_WIND             0
DE_TEMP             0
FR_TEMP             0
GAS_RET             0
COAL_RET            0
CARBON_RET          0
dtype: int64
ID        0
TARGET    0
dtype: int64
ID                  0
DAY_ID              0
COUNTRY             0
DE_CONSUMPTION      0
FR_CONSUMPTION      0
DE_FR_EXCHANGE      0
FR_DE_EXCHANGE      0
DE_NET_EXPORT       0
FR_N

In [6]:
y_train

,ID,TARGET
0,1054,0.028313
1,2049,-0.112516
2,1924,-0.180840
3,297,-0.260356
4,1101,-0.071733
...,...,...
1489,459,-0.172597
1490,1674,-0.063546
1491,748,0.151797
1492,1454,-0.640917


In [7]:
y_train_2= y_train['TARGET']

In [8]:
base_regressor = LinearRegression()

In [9]:
x_train_2 = x_train.drop(['COUNTRY'], axis=1).fillna(0)

In [10]:
x_test_2= x_test.drop(['COUNTRY'],axis=1).fillna(0)

In [11]:
y_random_2= y_random['TARGET']

In [12]:
adaboost_regressor_lr = AdaBoostRegressor(base_regressor, n_estimators=50, random_state=42)
adaboost_regressor_lr.fit(x_train_2, y_train_2)
predictions = adaboost_regressor_lr.predict(x_test_2)
mse = mean_squared_error(y_random_2, predictions)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 1.5294019636055427


In [13]:
base_regressor_2 =  RandomForestRegressor()

In [14]:
adaboost_regressor_rf = AdaBoostRegressor(base_regressor_2, n_estimators=50, random_state=42)
adaboost_regressor_rf.fit(x_train_2, y_train_2)
predictions_2 = adaboost_regressor_rf.predict(x_test_2)
mse_rf = mean_squared_error(y_random_2, predictions_2)
print(f'Mean Squared Error for Random Forest: {mse_rf}')

Mean Squared Error for Random Forest: 1.2629790366025222


In [15]:
base_regressor_3 = DecisionTreeRegressor()

In [16]:
adaboost_regressor_dt = AdaBoostRegressor(base_regressor_3, n_estimators=50, random_state=42)
adaboost_regressor_dt.fit(x_train_2, y_train_2)
predictions_3 = adaboost_regressor_dt.predict(x_test_2)
mse_dt = mean_squared_error(y_random_2, predictions_3)
print(f'Mean Squared Error for Decision Tree: {mse_dt}')

Mean Squared Error for Decision Tree: 1.113849169450042


In [17]:
train_lr_output= adaboost_regressor_lr.predict(x_train_2)
train_rf_output= adaboost_regressor_rf.predict(x_train_2)
train_dt_output= adaboost_regressor_dt.predict(x_train_2)


In [18]:
from sklearn.model_selection import cross_val_score
combined_outputs = np.column_stack((train_lr_output, train_rf_output, train_dt_output))
combined_lr= LinearRegression()
combined_lr.fit(combined_outputs, y_train_2)
ensemble = combined_lr.predict(combined_outputs)
cross_val_scores = cross_val_score(combined_lr, combined_outputs, y_train_2, cv=5, scoring='neg_mean_squared_error')
ensemble_correlation = spearmanr(ensemble, y_train_2).correlation

print('Spearman correlation for the ensemble on the train set: {:.1f}%'.format(100 * ensemble_correlation))
print('Cross-Validation MSE scores:', -cross_val_scores)

Spearman correlation for the ensemble on the train set: 98.6%
Cross-Validation MSE scores: [0.00452685 0.00639209 0.00542288 0.00555467 0.00836539]


In [19]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
lr = LinearRegression()
lr.fit(x_test_2, y_random_2)
output_test_lr = lr.predict(x_test_2)


model = AdaBoostRegressor()

# Define custom scorer for maximizing Spearman correlation
spearman_scorer = make_scorer(lambda y_true, y_pred: spearmanr(y_true, y_pred).correlation, greater_is_better=True)

# Define hyperparameters for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200, 1000],
    'learning_rate': [0.01, 0.1, 0.2],

}

# Create GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=5, scoring=spearman_scorer)

# Fit the model
grid_search.fit(x_test_2, y_random_2)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Make predictions using the best
output_test = best_model.predict(x_test_2)

# Stack the predictions of individual models as additional features
ensemble_input_test = np.column_stack((output_test_lr, output_test))

# Ensemble model (Linear Regression)
ensemble_lr = LinearRegression()
ensemble_lr.fit(ensemble_input_test, y_random_2)

# Make predictions using the ensemble model
output_test_ensemble = ensemble_lr.predict(ensemble_input_test)

# Cross-validation for Ensemble model using MSE scoring
ensemble_cv_scores = -cross_val_score(ensemble_lr, ensemble_input_test, y_random_2, cv=5, scoring='neg_mean_squared_error')

# Calculate Spearman correlation for the ensemble on the test set
ensemble_correlation_test = spearmanr(output_test_ensemble, y_random_2).correlation
print('Cross-Validation MSE scores:', ensemble_cv_scores)

print('Spearman correlation for the ensemble on the test set: {:.1f}%'.format(100 * ensemble_correlation_test))
print('Best hyperparameters for AdaBoost:', grid_search.best_params_)

Cross-Validation MSE scores: [0.83563109 0.89375151 0.7038503  0.9285527  0.87960461]
Spearman correlation for the ensemble on the test set: 42.7%
Best hyperparameters for AdaBoost: {'learning_rate': 0.2, 'n_estimators': 50}


In [20]:
rf_model = RandomForestRegressor()

# Define hyperparameters for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],

}

# Create GridSearchCV object for Random Forest
grid_search_rf = GridSearchCV(rf_model, rf_param_grid, cv=5, scoring=spearman_scorer, n_jobs=-1)
grid_search_rf.fit(x_test_2, y_random_2)
best_rf_model = grid_search_rf.best_estimator_
output_test_rf = best_rf_model.predict(x_test_2)



# Define hyperparameters for Ada
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],

}

# Create GridSearchCV object for Ada
grid_search = GridSearchCV(model, param_grid, cv=5, scoring=spearman_scorer, n_jobs=-1)
grid_search.fit(x_test_2, y_random_2)
best_model = grid_search.best_estimator_
output_test = best_model.predict(x_test_2)

# Stack the predictions of individual models as additional features
ensemble_input_test = np.column_stack((output_test_rf, output_test))

# Ensemble model (Linear Regression)
ensemble_lr = LinearRegression()
ensemble_lr.fit(ensemble_input_test, y_random_2)

# Make predictions using the ensemble model
output_test_ensemble = ensemble_lr.predict(ensemble_input_test)

# Calculate Spearman correlation for the ensemble on the test set
ensemble_correlation_test = spearmanr(output_test_ensemble, y_random_2).correlation
ensemble_cv_scores = -cross_val_score(ensemble_lr, ensemble_input_test, y_random_2, cv=5, scoring='neg_mean_squared_error')
print('Cross-Validation MSE scores:', ensemble_cv_scores)

print('Spearman correlation for the ensemble on the test set: {:.1f}%'.format(100 * ensemble_correlation_test))
print('Best hyperparameters for Random Forest:', grid_search_rf.best_params_)
print('Best hyperparameters for Gradient Boosting:', grid_search.best_params_)

Cross-Validation MSE scores: [0.10682144 0.09191417 0.07576489 0.09591167 0.10432431]
Spearman correlation for the ensemble on the test set: 95.7%
Best hyperparameters for Random Forest: {'max_depth': 20, 'n_estimators': 50}
Best hyperparameters for Gradient Boosting: {'learning_rate': 0.01, 'n_estimators': 50}


In [21]:
dt_model = DecisionTreeRegressor()

# Define hyperparameters for Decision Tree
dt_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],

}

# Create GridSearchCV object for Decision Tree
grid_search_dt = GridSearchCV(dt_model, dt_param_grid, cv=5, scoring=spearman_scorer, n_jobs=-1)
grid_search_rf.fit(x_test_2, y_random_2)
best_rf_model = grid_search_rf.best_estimator_
output_test_rf = best_rf_model.predict(x_test_2)



# Define hyperparameters
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],

}

# Create GridSearchCV object for Ada
grid_search = GridSearchCV(model, param_grid, cv=5, scoring=spearman_scorer, n_jobs=-1)
grid_search.fit(x_test_2, y_random_2)
best_model = grid_search.best_estimator_
output_test = best_model.predict(x_test_2)

# Stack the predictions of individual models as additional features
ensemble_input_test = np.column_stack((output_test_rf, output_test))

# Ensemble model (Linear Regression)
ensemble_lr = LinearRegression()
ensemble_lr.fit(ensemble_input_test, y_random_2)

# Make predictions using the ensemble model
output_test_ensemble = ensemble_lr.predict(ensemble_input_test)

# Calculate Spearman correlation for the ensemble on the test set
ensemble_correlation_test = spearmanr(output_test_ensemble, y_random_2).correlation
ensemble_cv_scores = -cross_val_score(ensemble_lr, ensemble_input_test, y_random_2, cv=5, scoring='neg_mean_squared_error')
print('Cross-Validation MSE scores:', ensemble_cv_scores)


print('Spearman correlation for the ensemble on the test set: {:.1f}%'.format(100 * ensemble_correlation_test))
print('Best hyperparameters for Random Forest:', grid_search_rf.best_params_)
print('Best hyperparameters for Ada Boost', grid_search.best_params_)

Cross-Validation MSE scores: [0.12034905 0.09476644 0.06785871 0.09991214 0.08018827]
Spearman correlation for the ensemble on the test set: 95.5%
Best hyperparameters for Random Forest: {'max_depth': 20, 'n_estimators': 50}
Best hyperparameters for Ada Boost {'learning_rate': 0.2, 'n_estimators': 100}
